In [3]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Доступные модели, поддерживающие метод 'generateContent':\n")
    
    # Ищем модели, которые умеют генерировать контент (текст, картинки)
    for model in genai.list_models():
        if 'generateContent' in model.supported_generation_methods:
            print(f"- {model.name}")
else:
    print("Не найден ключ Google API в .env файле.")

Доступные модели, поддерживающие метод 'generateContent':

- models/gemini-2.5-pro-preview-03-25
- models/gemini-2.5-flash-preview-05-20
- models/gemini-2.5-flash
- models/gemini-2.5-flash-lite-preview-06-17
- models/gemini-2.5-pro-preview-05-06
- models/gemini-2.5-pro-preview-06-05
- models/gemini-2.5-pro
- models/gemini-2.0-flash-exp
- models/gemini-2.0-flash
- models/gemini-2.0-flash-001
- models/gemini-2.0-flash-lite-001
- models/gemini-2.0-flash-lite
- models/gemini-2.0-flash-lite-preview-02-05
- models/gemini-2.0-flash-lite-preview
- models/gemini-2.0-pro-exp
- models/gemini-2.0-pro-exp-02-05
- models/gemini-exp-1206
- models/gemini-2.0-flash-thinking-exp-01-21
- models/gemini-2.0-flash-thinking-exp
- models/gemini-2.0-flash-thinking-exp-1219
- models/gemini-2.5-flash-preview-tts
- models/gemini-2.5-pro-preview-tts
- models/learnlm-2.0-flash-experimental
- models/gemma-3-1b-it
- models/gemma-3-4b-it
- models/gemma-3-12b-it
- models/gemma-3-27b-it
- models/gemma-3n-e4b-it
- models

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from IPython.display import display
from PIL import Image
import google.generativeai as genai

# --- 1. Загрузка ключа и конфигурация API Google ---
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

key_looks_valid = False
if GOOGLE_API_KEY and GOOGLE_API_KEY != "AIza...":
    print("✅ Ключ Google API найден. Конфигурируем клиент.")
    try:
        genai.configure(api_key=GOOGLE_API_KEY)
        key_looks_valid = True
    except Exception as e:
        print(f"❌ Не удалось сконфигурировать клиент Google: {e}")
else:
    print("❌ Ключ Google API не найден или не изменен. Проверьте ваш .env файл.")

# --- 2. Основной цикл (запускается, если ключ верный) ---
if key_looks_valid:
    # --- Конфигурация ---
    # Для начала протестируем одну, самую мощную модель
    MODEL_1_ID = "models/gemini-2.0-flash"
    
    # Если захотите добавить вторую модель, например, от DeepSeek,
    # нужно будет получить их ключ и написать похожую функцию.
    
    IMG_DIR = "img"
    PROMPT = "Скажи, изображена ли на данном фото кошка породы \"Британская короткошерстная\", дай коротки ответ \"да\" или \"нет\". Опиши одним словом каждый признак, который убедил тебя в правильности или неправильности ответа. Если это не британская короткошерстная кошка, скажи какая это порода."

    # --- Функция для запроса к Gemini ---
    def query_gemini_vision(image_path, prompt):
        try:
            # Инициализируем модель
            model = genai.GenerativeModel(MODEL_1_ID)
            
            # Открываем изображение
            img = Image.open(image_path)
            
            # Отправляем промпт и изображение.
            # Библиотека сама обрабатывает кодирование и отправку.
            response = model.generate_content([prompt, img])
            
            return response.text

        except Exception as e:
            return f"Error: {type(e).__name__}, {e}"

    # --- Цикл обработки ---
    try:
        image_files = [f for f in os.listdir(IMG_DIR) if f.endswith(('.jpg', '.jpeg', '.png'))]
        print(f"\nНайдено {len(image_files)} изображений для обработки.")
    except FileNotFoundError:
        print(f"Ошибка: Директория '{IMG_DIR}' не найдена.")
        image_files = []

    results_data = []
    if image_files:
        print(f"Начинаем обработку с использованием модели: {MODEL_1_ID}")
        for filename in tqdm(image_files, desc="Обработка изображений"):
            image_path = os.path.join(IMG_DIR, filename)
            
            # Пока получаем ответ только от одной модели для простоты
            response_1 = query_gemini_vision(image_path, PROMPT)
            
            results_data.append({
                "filename": filename,
                "model_1_response": response_1,
                "model_2_response": "N/A" # Заглушка для второй модели
            })
        
        df_results = pd.DataFrame(results_data)
        
        print("\nСбор данных завершен. Вот первые 5 результатов:")
        display(df_results.head())
        
        output_csv_path = "gemini_vision_results_2.csv"
        df_results.to_csv(output_csv_path, index=False, encoding='utf-8')
        print(f"✅ Результаты сохранены в файл: {output_csv_path}")
        
    else:
        print("Нет файлов для обработки.")
else:
    print("\nОсновной процесс не был запущен.")

✅ Ключ Google API найден. Конфигурируем клиент.

Найдено 30 изображений для обработки.
Начинаем обработку с использованием модели: models/gemini-2.0-flash


Обработка изображений: 100%|██████████| 30/30 [00:33<00:00,  1.12s/it]


Сбор данных завершен. Вот первые 5 результатов:


,filename,model_1_response,model_2_response
0,brit1.jpg,"да\nГолова Круглая, морда короткая.",N/A
1,brit10.jpg,да\n* **Голова:** Круглая\n* **Шерсть:** Корот...,N/A
2,brit2.jpg,Да.\n\n* Шерсть\n* Морда\n* Глаза,N/A
3,brit3.jpg,да\nГолова: круглая\nГлаза: круглые\nШерсть: к...,N/A
4,brit4.jpg,да\nпризнак: шерсть,N/A


✅ Результаты сохранены в файл: gemini_vision_results_2.csv
